In [1]:
import os
import re
import json
import requests
import pandas as pd
import geopandas as gpd
from bs4 import BeautifulSoup as BS
from bs4 import NavigableString as NS

10000-20000: Antwerpen  
11000-12000: Admin Antwerpen  
12000-13000: Admin Mechelen  
13000-14000: Admin Turnhout  

21000-22000: Brussel  
22000-24000: Vlaams-Brabant  
22000-23000: Admin Halle-Vilvoorde  
23000-24000: Admin Leuven  

30000-40000: West-Vlaanderen  
40000-50000: Oost-Vlaanderen  
70000-80000: Limburg  

In [3]:
df = pd.read_csv("REFNIS_2019.csv", encoding='latin1', sep=";")
df.drop(["Code INS","Entités administratives","Langue"], axis=1, inplace=True)
df.drop([f"Unnamed: {idx}" for idx in range(6,15)], axis=1, inplace=True)

admin = df[df["Code NIS"].apply(lambda x: 10000<=x<20000) & df["Code NIS"].apply(lambda x: x%1000 is 0)]

drop = df[df["Code NIS"].apply(lambda x: 10000<=x<20000) & df["Code NIS"].apply(lambda x: x%1000 is 0)]
province = df[df["Code NIS"].apply(lambda x: 10000<=x<20000)]
province = province.sort_values(["Administratieve eenheden"]) # Sort by name
province[province["Administratieve eenheden"].apply(lambda x: x[0] == "A")] # Starts with letter
pass

In [4]:
class NIS:
    def __init__(self, place:str, code:int):
        self.place = place
        self.code = code
        
    @classmethod
    def from_place(cls, place:str):
        pass
    
    def initialize_dictionary():
        province_nis_range = {
        "Antwerpen":       lambda x: 10000 <=x <20000,
        "Brussel":         lambda x: 21000 <=x <22000,
        "Vlaams-Brabant":  lambda x: 22000 <=x <24000,
        "West-Vlaanderen": lambda x: 30000 <=x <40000,
        "Oost-Vlaanderen": lambda x: 40000 <=x <50000,
        "Limburg":         lambda x: 70000 <=x <80000}

        nis_lookup = {}
        for province, nis_range in province_nis_range.items():
            dataframe = df[df["Code NIS"].apply(nis_range) & df["Code NIS"].apply(lambda x: x%1000 is not 0)]
            dataframe = dataframe.sort_values("Administratieve eenheden")
            nis_lookup[province] = dict(zip(
                (item[1] for item in dataframe["Administratieve eenheden"].items()),
                (item[1] for item in dataframe["Code NIS"].items())))

    def print_nis_lookup():
        for province, collection in nis_lookup.items():
            print(f"{province}")
            for place, nis in collection.items():
                print(f"\t{place} : {nis}")
                
    

NIS.print_nis_lookup()

NameError: name 'nis_lookup' is not defined

In [435]:
NIS = nis_lookup["Antwerpen"]["Duffel"]
print(NIS)
print(type(NIS))
#url = f"https://eservices.minfin.fgov.be/myminfin-rest/cadastral-plan/cadastralPlan/2020/12009/72"
#req = requests.get(url)

12009
<class 'int'>


In [440]:
from io import BytesIO
from zipfile import ZipFile

def get_shapefiles_from_nis_code(file_names:[], nis:NisCode):
    url = f"https://eservices.minfin.fgov.be/myminfin-rest/cadastral-plan/cadastralPlan/2020/{nis.code}/72"
    with ZipFile(BytesIO(requests.get(url).content)) as package:
        for content in package.namelist():
            if re.match(f".*({file_type}).*", content):
                if not os.path.exists("./Test/"):
                    os.mkdir("./Test/")
                with open(f"./Test/{content}", "wb") as file:
                    file.write(package.read(content))

#get_shapefiles_from_nis_code(["CaBu","ReBu","CaBl"], NIS)

#shape = gpd.read_file(f"./Test/Bli_EaZo.shp")